In [ ]:
import cv2
import threading
import time

# Función que aplica el filtro de escala de grises a una sección de la imagen
def aplicar_filtro(seccion, inicio, fin, hilo_num):
    print(f'Hilo {hilo_num}: Procesando sección desde fila {inicio} hasta fila {fin - 1}')
    inicio_tiempo = time.time()
    for i in range(inicio, fin):
        for j in range(len(seccion[i])):
            pixel = seccion[i][j]
            gris = sum(pixel) // 3  # Escala de grises
            seccion[i][j] = [gris, gris, gris]
    fin_tiempo = time.time()
    print(f'Hilo {hilo_num}: Sección procesada en {fin_tiempo - inicio_tiempo:.2f} segundos')

# Función que divide la imagen en secciones y aplica el filtro en paralelo
def procesar_imagen(imagen, num_hilos):
    hilo_secciones = []
    alto, ancho, _ = imagen.shape
    seccion_alto = alto // num_hilos
    
    for i in range(num_hilos):
        inicio = i * seccion_alto
        fin = (i + 1) * seccion_alto if i < num_hilos - 1 else alto
        seccion = imagen[inicio:fin]
        hilo = threading.Thread(target=aplicar_filtro, args=(seccion, 0, len(seccion), i))
        hilo.start()
        hilo_secciones.append(hilo)

    # Esperar a que todos los hilos terminen
    for hilo in hilo_secciones:
        hilo.join()

# Cargar la imagen
imagen = cv2.imread('imagen.jpg')

# Especificar el número de hilos que deseas usar para el procesamiento
num_hilos = 4

print(f'Iniciando procesamiento con {num_hilos} hilos...')
inicio_total = time.time()

# Procesar la imagen en paralelo
procesar_imagen(imagen, num_hilos)

fin_total = time.time()
print(f'Tiempo total de procesamiento: {fin_total - inicio_total:.2f} segundos')

# Mostrar la imagen procesada
cv2.imshow('Imagen Procesada', imagen)
cv2.waitKey(0)
cv2.destroyAllWindows()